In [44]:
import numpy as np

numpy.concatenate()
- Công dụng: Dùng để nối các mảng lại với nhau theo một trục (axis) xác định.

numpy.c_[]
- Công dụng: chủ yếu được dùng để thêm cột vào một mảng 2D hoặc chuyển một mảng 1D thành dạng 2D bằng cách nối theo chiều dọc (axis 1).

### Câu 7: Polynomial feature cho hàm đơn biến

In [45]:
def create_polynomial_features(x, degree=2):
    """ 
    Creates the polynomial features
    Args:
        x: A array tensor for the data.
        degree: An integer for the degree of the generated polynomial function.
    """
    
    x_new = x
    for d in range(2, degree +1):
        # Nối các mảng 2D theo chiều dọc (như concatenate với axis=1)
        x_new = np.c_[x_new, np.power(x, d)]
    return x_new

x = np.array([[1], 
              [2], 
              [3]])
print(create_polynomial_features(x, 2))

[[1 1]
 [2 4]
 [3 9]]


### Câu 8 : polynomial feature đa biến

#### Cách cá nhân

In [46]:
def create_polynomial_features(x:np.ndarray, degree=2):
    """ 
    Creates the polynomial features
    Args:
        x: A array tensor for the data.
        degree: An integer for the degree of the generated polynomial function.
    """

    x_new = x
    for d in range(2, degree + 1):
        for i_feat in range(x.shape[1] - 1):
            # Nối các mảng 2D theo chiều dọc (như concatenate với axis=1)
            x_new = np.insert(x_new, i_feat + 1, np.power(x[:, i_feat], d), axis=1)
        
        x_new = np.c_[x_new, np.power(x[:, -1], d)]
    return x_new


x = np.array([[1, 2],
              [2, 3],
              [3, 4]])
print(create_polynomial_features(x, 2))

[[ 1  1  2  4]
 [ 2  4  3  9]
 [ 3  9  4 16]]


In [47]:
x = np.array([[1, 2],
              [2, 3],
              [3, 4]])
x_new = x 
x_new = np.insert(x, 1, np.power(x[:, 0], 2), axis=1)
print(x_new)

[[1 1 2]
 [2 4 3]
 [3 9 4]]


#### Cách theo đề

In [49]:
def create_polynomial_features(X, degree=2):
    """Creates the polynomial features.
    
    Args:
        X: A numpy array for the data.
        degree: An integer for the degree of the generated polynomial function.
    """
    X_mem = []
    for X_sub in X.T:
        X_sub = X_sub.T
        X_new = X_sub
        for d in range(2, degree + 1):
            X_new = np.c_[X_new, np.power(X_sub, d)]
        X_mem.extend(X_new.T)
    print(X_mem)
    return np.c_[X_mem].T


x = np.array([[1, 2],
              [2, 3],
              [3, 4]])

print(create_polynomial_features(x, 2))

[array([1, 2, 3]), array([1, 4, 9]), array([2, 3, 4]), array([ 4,  9, 16])]
[[ 1  1  2  4]
 [ 2  4  3  9]
 [ 3  9  4 16]]
